In [1]:
import sys

sys.path.append("../")

In [2]:
from omegaconf import OmegaConf, DictConfig
from dacite import from_dict, Config as DaciteConfig
from pprint import pprint

import jax
from jax import lax
from jax.sharding import Mesh, NamedSharding, PartitionSpec as P
import jax.numpy as jnp
from flax import nnx
from flax.nnx import initializers
import optax
from functools import partial
import typing as tp

In [3]:
from xlstm_jax import xLSTMLMModel, xLSTMLMModelConfig

In [4]:
# create new model
xlstm_cfg = """ 
vocab_size: 128
context_length: 32      
num_blocks: 2 #!
embedding_dim: 64 #!
tie_weights: false
mlstm_block:
  mlstm:
    conv1d_kernel_size: 4
    qkv_proj_blocksize: 4
    num_heads: 4
"""
cfg = OmegaConf.create(xlstm_cfg)
cfg = from_dict(
    data_class=xLSTMLMModelConfig,
    data=OmegaConf.to_container(cfg),
    config=DaciteConfig(strict=True),
)

In [5]:
pprint(cfg)

xLSTMLMModelConfig(mlstm_block=mLSTMBlockConfig(mlstm=mLSTMLayerConfig(proj_factor=2.0,
                                                                       round_proj_up_dim_up=True,
                                                                       round_proj_up_to_multiple_of=64,
                                                                       _proj_up_dim=128,
                                                                       conv1d_kernel_size=4,
                                                                       qkv_proj_blocksize=4,
                                                                       num_heads=4,
                                                                       embedding_dim=64,
                                                                       bias=False,
                                                                       dropout=0.0,
                                                                       context_length=32,
     

In [6]:
from training.utils.array import create_mesh


In [7]:
mesh = create_mesh((1, 1), ("dp", "tp"))
rngs = nnx.Rngs(123)
dtype = jnp.bfloat16
param_dtype = jnp.float32

In [8]:
model = xLSTMLMModel(
    cfg,
    mesh=mesh,
    rngs=rngs,
    dtype=dtype,
    param_dtype=param_dtype,
)

In [9]:
nnx.display(model)

In [10]:
x_in = jax.random.randint(
    rngs(),
    shape=(2, 32),
    minval=0,
    maxval=cfg.vocab_size,
)

In [11]:
y = model(x_in)